In [1]:
%matplotlib inline

import numpy as np
import scipy.linalg
import scipy.sparse.linalg
import matplotlib.pyplot as plt
import time
from numba import jit

In [2]:
@jit(nopython=True)
def build_pascal_orig(lc,nc):
    cnkc = np.zeros((lc,nc),dtype=np.int64)
    for i in range(1,lc+1):
        cnkc[i-1,0] = 1
    for i in range(1,lc+1):
        for j in range(2,nc+1):
            cnkc[i-1,j-1] = 0
    for in1 in range(2,lc+1):
        cnkc[in1-1,1] = np.sum(cnkc[in1-2,0:2])
        if nc-1 > 1:
            for in2 in range(1,nc+1):
                cnkc[in1-1,in2-1] = np.sum(cnkc[in1-2,0:in2])
    return cnkc, cnkc[lc-1,nc-1]

@jit(nopython=True)
def build_pascal(lc,nc):
    cnkc = np.zeros((lc,nc),dtype=np.int64)
    cnkc[0,0] = 1
    for in1 in range(2,lc+1):
        for in2 in range(1,nc+1):
            cnkc[in1-1,in2-1] = np.sum(cnkc[in1-2,0:in2])
#            print(cnkc)
    return cnkc, cnkc[lc-1,nc-1]

@jit(nopython=True)
def build_pascal_truncated(lc,nc,nmaxp):
    cnkct = np.zeros((lc,nc),dtype=np.int64)
    cnkct[0,0] = 1
    for in1 in range(2,lc+1):
        for in2 in range(1,nmaxp+1):
            cnkct[in1-1,in2-1] = np.sum(cnkct[in1-2,0:in2])
        for in2 in range(nmaxp+1,nc+1):
            cnkct[in1-1,in2-1] = np.sum(cnkct[in1-2,in2-nmaxp:in2])
#            print(cnkct)
    return cnkct, cnkct[lc-1,nc-1]


In [3]:
Ns = 4 # number of sites
Na = 4 # number of atoms
nmax = 2 # |n> = |0>, |1>, ... |nmax> ((nmax+1) states)
lc = Ns + 1 # number of sites + 1
nc = Na + 1  # number of atoms + 1
nmaxp = nmax + 1 # nmax + 1, size of local Hilbert space

cnkc_orig, jmax_orig = build_pascal_orig(lc,nc)
print(cnkc_orig)
print(jmax_orig)

cnkc, jmax = build_pascal(lc,nc)
print(cnkc)
print(jmax)

cnkct, jmaxt = build_pascal_truncated(lc,nc,nmaxp)
print(cnkct)
print(jmaxt)

[[ 1  0  0  0  0]
 [ 1  1  1  1  1]
 [ 1  2  3  4  5]
 [ 1  3  6 10 15]
 [ 1  4 10 20 35]]
35
[[ 1  0  0  0  0]
 [ 1  1  1  1  1]
 [ 1  2  3  4  5]
 [ 1  3  6 10 15]
 [ 1  4 10 20 35]]
35
[[ 1  0  0  0  0]
 [ 1  1  1  0  0]
 [ 1  2  3  2  1]
 [ 1  3  6  7  6]
 [ 1  4 10 16 19]]
19


In [4]:
# Returns the position ind of the many body state bi
# we do not need explicit input "nmaxp"
@jit(nopython=True)
def b2in(bi,ind,cnkc,lc,nc): # basis to index
    ind[0] = 1 ## start from 1
    num = nc
    for ind_site in range(1,lc-1): # site loop
        numb = bi[ind_site-1]
        ind[0] += np.sum(cnkc[lc-1-ind_site,num-numb:num])
        #print(ind_site,num-1,numb,cnkc[lc-1-ind_site,num-numb:num])
        num -= numb
    return 0

In [5]:
Ns = 4 # number of sites
Na = 4 # number of atoms
nmax = 2 # |n> = |0>, |1>, ... |nmax> ((nmax+1) states)
lc = Ns + 1 # number of sites + 1
nc = Na + 1  # number of atoms + 1
nmaxp = nmax + 1 # nmax + 1, size of local Hilbert space
cnkc, jmax = build_pascal_truncated(lc,nc,nmaxp)
print(cnkc)
print(jmax)
print()

#bi = np.zeros(lc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)

bi = np.array([0,0,2,2])
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

bi = np.array([1,1,1,1])
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

bi = np.array([2,2,0,0])
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

[[ 1  0  0  0  0]
 [ 1  1  1  0  0]
 [ 1  2  3  2  1]
 [ 1  3  6  7  6]
 [ 1  4 10 16 19]]
19

[0 0 2 2] 1
[1 1 1 1] 10
[2 2 0 0] 19


In [6]:
# Returns the position ind of the many body state bi_short
# we do not need explicit input "nmaxp"
@jit(nopython=True)
def bshort2in(bis,ind,cnkc,lc,nc): # basis to index
    ind[0] = 1 ## start from 1
    num = 2
    for ind_atom in range(1,nc): # atom loop
        ind_site = bis[ind_atom-1]
        ind[0] += cnkc[ind_site-1,num-1]
        #print(ind_atom,ind_site,num,cnkc[ind_site-1,num-1],ind[0])
        num += 1
    return 0

In [7]:
Ns = 4 # number of sites
Na = 4 # number of atoms
nmax = 2 # |n> = |0>, |1>, ... |nmax> ((nmax+1) states)
lc = Ns + 1 # number of sites + 1
nc = Na + 1  # number of atoms + 1
nmaxp = nmax + 1 # nmax + 1, size of local Hilbert space
cnkc, jmax = build_pascal_truncated(lc,nc,nmaxp)
print(cnkc)
print(jmax)
print()

#bi = np.zeros(lc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)

bi = np.array([1,1,2,2])
bshort2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

bi = np.array([1,2,3,4])
bshort2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

bi = np.array([3,3,4,4])
bshort2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

[[ 1  0  0  0  0]
 [ 1  1  1  0  0]
 [ 1  2  3  2  1]
 [ 1  3  6  7  6]
 [ 1  4 10 16 19]]
19

[1 1 2 2] 1
[1 2 3 4] 10
[3 3 4 4] 19


In [8]:
# Returns the many body state bi at position ind
# we do not need explicit input "nmaxp"
@jit(nopython=True)
def in2b(bi,ind,cnkc,lc,nc): # index to basis
    ind_i = ind[0] - 1 ## ind[0] runs from 1 to jmax=cnkc[ind_lc-1,ind_nc-1]
    bi[:] = 0
    ind_L = lc - 1
    ind_N = nc
    while ind_N > 1: # atom loop
        if ind_i >= cnkc[ind_L-1,ind_N-1]: # condition for site
            ind_i -= cnkc[ind_L-1,ind_N-1]
            bi[lc-ind_L-1] += 1
            ind_N -= 1
        else:
            ind_L -= 1
    return 0

In [9]:
Ns = 4 # number of sites
Na = 4 # number of atoms
nmax = 2 # |n> = |0>, |1>, ... |nmax> ((nmax+1) states)
lc = Ns + 1 # number of sites + 1
nc = Na + 1  # number of atoms + 1
nmaxp = nmax + 1 # nmax + 1, size of local Hilbert space
cnkc, jmax = build_pascal_truncated(lc,nc,nmaxp)
print(cnkc)
print(jmax)
print()

bi = np.zeros(lc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)

ind[0] = 1
in2b(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

ind[0] = 10
in2b(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

ind[0] = 19
in2b(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

[[ 1  0  0  0  0]
 [ 1  1  1  0  0]
 [ 1  2  3  2  1]
 [ 1  3  6  7  6]
 [ 1  4 10 16 19]]
19

[0 0 2 2] 1
[1 1 1 1] 10
[2 2 0 0] 19


In [10]:
# Returns the many body state bi_short at position ind
# we do not need explicit input "nmaxp"
@jit(nopython=True)
def in2bshort(bis,ind,cnkc,lc,nc): # index to basis short
    ind_i = ind[0] - 1 ## ind[0] runs from 1 to jmax=cnkc[ind_lc-1,ind_nc-1]
    bis[:] = 0
    ind_L = lc - 1
    ind_N = nc
    while ind_N > 1: # atom loop
        if ind_i >= cnkc[ind_L-1,ind_N-1]: # condition for site
            ind_i -= cnkc[ind_L-1,ind_N-1]
            bis[ind_N-2] = ind_L
            ind_N -= 1
        else:
            ind_L -= 1
    return 0

In [11]:
Ns = 4 # number of sites
Na = 4 # number of atoms
nmax = 2 # |n> = |0>, |1>, ... |nmax> ((nmax+1) states)
lc = Ns + 1 # number of sites + 1
nc = Na + 1  # number of atoms + 1
nmaxp = nmax + 1 # nmax + 1, size of local Hilbert space
cnkc, jmax = build_pascal_truncated(lc,nc,nmaxp)
print(cnkc)
print(jmax)
print()

bis = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)

ind[0] = 1
in2bshort(bis,ind,cnkc,lc,nc)
print(bis,ind[0])

ind[0] = 10
in2bshort(bis,ind,cnkc,lc,nc)
print(bis,ind[0])

ind[0] = 19
in2bshort(bis,ind,cnkc,lc,nc)
print(bis,ind[0])

[[ 1  0  0  0  0]
 [ 1  1  1  0  0]
 [ 1  2  3  2  1]
 [ 1  3  6  7  6]
 [ 1  4 10 16 19]]
19

[1 1 2 2] 1
[1 2 3 4] 10
[3 3 4 4] 19


In [12]:
## np.searchsorted is better?
@jit(nopython=True)
def binary_search_orig(s,list_s,ls,le):
    bmin = ls; bmax = le
    while True:
        b = bmin + (bmax-bmin)//2
        if s < list_s[b-1]:
            bmax = b - 1
        elif list_s[b-1] < s:
            bmin = b + 1
        else:
            bmin = b
            return b, bmin
        if bmin > bmax:
            b = -1
            return b, bmin
    return b, bmin

@jit(nopython=True)
def binary_search(s,list_s,ls,le,side="left"):
    return np.searchsorted(list_s[ls-1:le],s,side=side)+1

In [13]:
# Returns the many body state bi_short at position ind
# using binary search
# we do not need explicit input "nmaxp"
@jit(nopython=True)
def in2bshort_bs(bis,ind,cnkc,lc,nc): # index to basis short
    ind_i = ind[0] ## ind[0] runs from 1 to jmax=cnkc[ind_lc-1,ind_nc-1]
    bis[:] = 0
    ind_site = lc
    for ind_atom in range(nc,1,-1): # atom loop
#        icnkc, icnkcmin = binary_search_orig(ind_i,cnkc[:ind_site,ind_atom-1],1,ind_site)
        icnkcmin = binary_search(ind_i,cnkc[:ind_site,ind_atom-1],1,ind_site)
        #print(ind_atom,ind_i,icnkc,icnkcmin,cnkc[:ind_site,ind_atom-1],cnkc[icnkcmin-2,ind_atom-1])
        ind_i -= cnkc[icnkcmin-2,ind_atom-1]
        bis[ind_atom-2] = icnkcmin-1
        ind_site = icnkcmin
    return 0

In [14]:
Ns = 4 # number of sites
Na = 4 # number of atoms
nmax = 2 # |n> = |0>, |1>, ... |nmax> ((nmax+1) states)
lc = Ns + 1 # number of sites + 1
nc = Na + 1  # number of atoms + 1
nmaxp = nmax + 1 # nmax + 1, size of local Hilbert space
cnkc, jmax = build_pascal_truncated(lc,nc,nmaxp)
print(cnkc)
print(jmax)
print()

bis = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)

ind[0] = 1
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0])

ind[0] = 10
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0])

ind[0] = 19
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0])

[[ 1  0  0  0  0]
 [ 1  1  1  0  0]
 [ 1  2  3  2  1]
 [ 1  3  6  7  6]
 [ 1  4 10 16 19]]
19

[1 1 2 2] 1
[1 2 3 4] 10
[3 3 4 4] 19


In [15]:
Ns = 4 # number of sites
Na = 4 # number of atoms
nmax = 2 # |n> = |0>, |1>, ... |nmax> ((nmax+1) states)
lc = Ns + 1 # number of sites + 1
nc = Na + 1  # number of atoms + 1
nmaxp = nmax + 1 # nmax + 1, size of local Hilbert space
cnkc, jmax = build_pascal_truncated(lc,nc,nmaxp)
print(cnkc)
print(jmax)
print()

bi = np.zeros(lc-1,dtype=np.int64)
bis = np.zeros(nc-1,dtype=np.int64)
bis_bs = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind_bi = np.zeros(1,dtype=np.int64)
ind_bis = np.zeros(1,dtype=np.int64)
ind_bis_bs = np.zeros(1,dtype=np.int64)

for i in range(1,jmax+1):
#for i in list(range(1,9))+list(range(jmax+1-8,jmax+1)):
    ind[0] = i
    in2b(bi,ind,cnkc,lc,nc)
    in2bshort(bis,ind,cnkc,lc,nc)
    in2bshort_bs(bis_bs,ind,cnkc,lc,nc)
    b2in(bi,ind_bi,cnkc,lc,nc)
    bshort2in(bis,ind_bis,cnkc,lc,nc)
    bshort2in(bis_bs,ind_bis_bs,cnkc,lc,nc)
    print(ind[0],bi,ind_bi[0]," ",ind[0],bis,ind_bis[0]," ",ind[0],bis_bs,ind_bis_bs[0])

[[ 1  0  0  0  0]
 [ 1  1  1  0  0]
 [ 1  2  3  2  1]
 [ 1  3  6  7  6]
 [ 1  4 10 16 19]]
19

1 [0 0 2 2] 1   1 [1 1 2 2] 1   1 [1 1 2 2] 1
2 [0 1 1 2] 2   2 [1 1 2 3] 2   2 [1 1 2 3] 2
3 [0 1 2 1] 3   3 [1 2 2 3] 3   3 [1 2 2 3] 3
4 [0 2 0 2] 4   4 [1 1 3 3] 4   4 [1 1 3 3] 4
5 [0 2 1 1] 5   5 [1 2 3 3] 5   5 [1 2 3 3] 5
6 [0 2 2 0] 6   6 [2 2 3 3] 6   6 [2 2 3 3] 6
7 [1 0 1 2] 7   7 [1 1 2 4] 7   7 [1 1 2 4] 7
8 [1 0 2 1] 8   8 [1 2 2 4] 8   8 [1 2 2 4] 8
9 [1 1 0 2] 9   9 [1 1 3 4] 9   9 [1 1 3 4] 9
10 [1 1 1 1] 10   10 [1 2 3 4] 10   10 [1 2 3 4] 10
11 [1 1 2 0] 11   11 [2 2 3 4] 11   11 [2 2 3 4] 11
12 [1 2 0 1] 12   12 [1 3 3 4] 12   12 [1 3 3 4] 12
13 [1 2 1 0] 13   13 [2 3 3 4] 13   13 [2 3 3 4] 13
14 [2 0 0 2] 14   14 [1 1 4 4] 14   14 [1 1 4 4] 14
15 [2 0 1 1] 15   15 [1 2 4 4] 15   15 [1 2 4 4] 15
16 [2 0 2 0] 16   16 [2 2 4 4] 16   16 [2 2 4 4] 16
17 [2 1 0 1] 17   17 [1 3 4 4] 17   17 [1 3 4 4] 17
18 [2 1 1 0] 18   18 [2 3 4 4] 18   18 [2 3 4 4] 18
19 [2 2 0 0] 19   19 [3

In [16]:
## binary search not efficient for small nmax?

@jit(nopython=True)
def calc_aiadj(bis2,bis,i,j,nmax): # assume i<j
    x = binary_search(i,bis,1,nmax) + np.sum(bis==i) - 1
    y = binary_search(j,bis,1,nmax) + np.sum(bis==j) - 1
    bis2[0:x-1] = bis[0:x-1]
    bis2[x-1:y-1] = bis[x:y]
    bis2[y-1] = j
    bis2[y:nmax] = bis[y:nmax]
    return 0

@jit(nopython=True)
def calc_adiaj(bis2,bis,i,j,nmax): # assume i<j
    x = binary_search(i,bis,1,nmax) + np.sum(bis==i)
    y = binary_search(j,bis,1,nmax)
    bis2[0:x-1] = bis[0:x-1]
    bis2[x-1] = i
    bis2[x:y] = bis[x-1:y-1]
    bis2[y:nmax] = bis[y:nmax]
    return 0

In [17]:
Ns = 4 # number of sites
Na = 4 # number of atoms
nmax = 2 # |n> = |0>, |1>, ... |nmax> ((nmax+1) states)
lc = Ns + 1 # number of sites + 1
nc = Na + 1  # number of atoms + 1
nmaxp = nmax + 1 # nmax + 1, size of local Hilbert space
cnkc, jmax = build_pascal_truncated(lc,nc,nmaxp)
print(cnkc)
print(jmax)
print()

bis = np.zeros(nc-1,dtype=np.int64)
bis2 = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind2 = np.zeros(1,dtype=np.int64)

for id in range(1,jmax+1):
    ind[0] = id
    in2bshort_bs(bis,ind,cnkc,lc,nc)
    print(bis,ind[0])
    for i in range(1,lc):
        for j in range(i+1,lc):
            if i in bis:
                calc_aiadj(bis2,bis,i,j,nc-1)
                bshort2in(bis2,ind2,cnkc,lc,nc)
                if np.sum(bis2==j) <= nmax:
                    coeff = np.sqrt(np.sum(bis==i)*(np.sum(bis==j)+1)) # \sqrt(n_i(nj+1))
                    print(i,j,bis2,ind2[0],coeff)
    print()
print()

[[ 1  0  0  0  0]
 [ 1  1  1  0  0]
 [ 1  2  3  2  1]
 [ 1  3  6  7  6]
 [ 1  4 10 16 19]]
19

[1 1 2 2] 1
1 3 [1 2 2 3] 3 1.4142135623730951
1 4 [1 2 2 4] 8 1.4142135623730951
2 3 [1 1 2 3] 2 1.4142135623730951
2 4 [1 1 2 4] 7 1.4142135623730951

[1 1 2 3] 2
1 2 [1 2 2 3] 3 2.0
1 3 [1 2 3 3] 5 2.0
1 4 [1 2 3 4] 10 1.4142135623730951
2 3 [1 1 3 3] 4 1.4142135623730951
2 4 [1 1 3 4] 9 1.0
3 4 [1 1 2 4] 7 1.0

[1 2 2 3] 3
1 3 [2 2 3 3] 6 1.4142135623730951
1 4 [2 2 3 4] 11 1.0
2 3 [1 2 3 3] 5 2.0
2 4 [1 2 3 4] 10 1.4142135623730951
3 4 [1 2 2 4] 8 1.0

[1 1 3 3] 4
1 2 [1 2 3 3] 5 1.4142135623730951
1 4 [1 3 3 4] 12 1.4142135623730951
3 4 [1 1 3 4] 9 1.4142135623730951

[1 2 3 3] 5
1 2 [2 2 3 3] 6 1.4142135623730951
1 4 [2 3 3 4] 13 1.0
2 4 [1 3 3 4] 12 1.0
3 4 [1 2 3 4] 10 1.4142135623730951

[2 2 3 3] 6
2 4 [2 3 3 4] 13 1.4142135623730951
3 4 [2 2 3 4] 11 1.4142135623730951

[1 1 2 4] 7
1 2 [1 2 2 4] 8 2.0
1 3 [1 2 3 4] 10 1.4142135623730951
1 4 [1 2 4 4] 15 2.0
2 3 [1 1 3 4] 9 1.0
2 4 

In [18]:
Ns = 4 # number of sites
Na = 4 # number of atoms
nmax = 2 # |n> = |0>, |1>, ... |nmax> ((nmax+1) states)
lc = Ns + 1 # number of sites + 1
nc = Na + 1  # number of atoms + 1
nmaxp = nmax + 1 # nmax + 1, size of local Hilbert space
cnkc, jmax = build_pascal_truncated(lc,nc,nmaxp)
print(cnkc)
print(jmax)
print()

bis = np.zeros(nc-1,dtype=np.int64)
bis2 = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind2 = np.zeros(1,dtype=np.int64)

for id in range(1,jmax+1):
    ind[0] = id
    in2bshort_bs(bis,ind,cnkc,lc,nc)
    print(bis,ind[0])
    for i in range(1,lc):
        for j in range(i+1,lc):
            if j in bis:
                calc_adiaj(bis2,bis,i,j,nc-1)
                bshort2in(bis2,ind2,cnkc,lc,nc)
                if np.sum(bis2==i) <= nmax:
                    coeff = np.sqrt((np.sum(bis==i)+1)*np.sum(bis==j)) # \sqrt((n_i+1)nj)
                    print(i,j,bis2,ind2[0],coeff)
    print()
print()

[[ 1  0  0  0  0]
 [ 1  1  1  0  0]
 [ 1  2  3  2  1]
 [ 1  3  6  7  6]
 [ 1  4 10 16 19]]
19

[1 1 2 2] 1

[1 1 2 3] 2
2 3 [1 1 2 2] 1 1.4142135623730951

[1 2 2 3] 3
1 2 [1 1 2 3] 2 2.0
1 3 [1 1 2 2] 1 1.4142135623730951

[1 1 3 3] 4
2 3 [1 1 2 3] 2 1.4142135623730951

[1 2 3 3] 5
1 2 [1 1 3 3] 4 1.4142135623730951
1 3 [1 1 2 3] 2 2.0
2 3 [1 2 2 3] 3 2.0

[2 2 3 3] 6
1 2 [1 2 3 3] 5 1.4142135623730951
1 3 [1 2 2 3] 3 1.4142135623730951

[1 1 2 4] 7
2 4 [1 1 2 2] 1 1.4142135623730951
3 4 [1 1 2 3] 2 1.0

[1 2 2 4] 8
1 2 [1 1 2 4] 7 2.0
1 4 [1 1 2 2] 1 1.4142135623730951
3 4 [1 2 2 3] 3 1.0

[1 1 3 4] 9
2 3 [1 1 2 4] 7 1.0
2 4 [1 1 2 3] 2 1.0
3 4 [1 1 3 3] 4 1.4142135623730951

[1 2 3 4] 10
1 2 [1 1 3 4] 9 1.4142135623730951
1 3 [1 1 2 4] 7 1.4142135623730951
1 4 [1 1 2 3] 2 1.4142135623730951
2 3 [1 2 2 4] 8 1.4142135623730951
2 4 [1 2 2 3] 3 1.4142135623730951
3 4 [1 2 3 3] 5 1.4142135623730951

[2 2 3 4] 11
1 2 [1 2 3 4] 10 1.4142135623730951
1 3 [1 2 2 4] 8 1.0
1 4 [1 2 2 3] 3 1.0


In [19]:
@jit(nopython=True)
def make_parameters_1d(lc,U,J):
    no_U = lc - 1
    val_U = U * np.ones(no_U,dtype=np.float64)
    no_J = lc - 1
    pair_J = np.zeros((2,no_J),dtype=np.int64)
    val_J = J * np.ones(no_J,dtype=np.float64)
    for i in range(no_J):
        pair_J[0,i] = i%no_J+1
        pair_J[1,i] = (i+1)%no_J+1
        if pair_J[0,i] > pair_J[1,i]: # assume i<j for pair (i,j)
            tmp = pair_J[0,i]
            pair_J[0,i] = pair_J[1,i]
            pair_J[1,i] = tmp
    return no_U, val_U, no_J, pair_J, val_J

In [20]:
@jit(nopython=True)
def make_full_hamiltonian(lv,Ham,cnkc,lc,nc,nmax,no_U,val_U,no_J,pair_J,val_J):
    bis = np.zeros(nc-1,dtype=np.int64)
    bis2 = np.zeros(nc-1,dtype=np.int64)
    ind = np.zeros(1,dtype=np.int64)
    ind2 = np.zeros(1,dtype=np.int64)
    for i in range(1,lv+1): # state loop
        ind[0] = i
        in2bshort_bs(bis,ind,cnkc,lc,nc)
        for j in range(1,no_U+1): # site loop
            nj = np.sum(bis==j)
            Ham[i-1,i-1] += 0.5 * val_U[j-1] * nj * (nj-1.0)
        for j in range(1,no_J+1): # bond loop
            f1 = pair_J[0,j-1] in bis
            f2 = pair_J[1,j-1] in bis
            if f1:
                calc_aiadj(bis2,bis,pair_J[0,j-1],pair_J[1,j-1],nc-1)
                bshort2in(bis2,ind2,cnkc,lc,nc)
                if np.sum(bis2==pair_J[1,j-1]) <= nmax:
                    Ham[i-1,ind2[0]-1] -= val_J[j-1] * \
                        np.sqrt(np.sum(bis==pair_J[0,j-1]) * (np.sum(bis==pair_J[1,j-1])+1))
#                    print("### 1 aiadj",bis2,bis,pair_J[0,j-1],pair_J[1,j-1],np.sqrt(np.sum(bis==pair_J[0,j-1]) * (np.sum(bis==pair_J[1,j-1])+1)))
            if f2:
                calc_adiaj(bis2,bis,pair_J[0,j-1],pair_J[1,j-1],nc-1)
                bshort2in(bis2,ind2,cnkc,lc,nc)
                if np.sum(bis2==pair_J[0,j-1]) <= nmax:
                    Ham[i-1,ind2[0]-1] -= val_J[j-1] * \
                        np.sqrt((np.sum(bis==pair_J[0,j-1])+1) * np.sum(bis==pair_J[1,j-1]))
#                    print("### 2 adiaj",bis2,bis,pair_J[0,j-1],pair_J[1,j-1],np.sqrt((np.sum(bis==pair_J[0,j-1])+1) * np.sum(bis==pair_J[1,j-1])))
    return 0

In [21]:
#@jit(nopython=True)
def calculate_1d_full_diag(Ns,Na,nmax,U,J):
    start = time.time()
#    Ns = 7 # number of sites
#    Na = 7 # number of atoms
#    nmax = 2 # |n> = |0>, |1>, ... |nmax> ((nmax+1) states)
    lc = Ns + 1 # number of sites + 1
    nc = Na + 1  # number of atoms + 1
    nmaxp = nmax + 1 # nmax + 1, size of local Hilbert space
    cnkc, jmax = build_pascal_truncated(lc,nc,nmaxp)
    print("# Ns,Na,nmax",Ns,Na,nmax)
    print("# cnkc",cnkc)
    print("# total Hilbert space size",jmax)
    end = time.time()
    print("## time build_pascal",end-start)
    print()

    start = time.time()
#    U = 10.0
#    J = 1.0
    no_U, val_U, no_J, pair_J, val_J = make_parameters_1d(lc,U,J)
    print("# no_U",no_U)
    print("# val_U",val_U)
    print("# no_J",no_J)
    print("# pair_J",pair_J)
    print("# val_J",val_J)
    end = time.time()
    print("## time make_parameters_1d",end-start)
    print()

    start = time.time()
    Ham = np.zeros((jmax,jmax),dtype=np.float64)
    make_full_hamiltonian(jmax,Ham,cnkc,lc,nc,nmax,no_U,val_U,no_J,pair_J,val_J)
#    print(Ham)
#    print(Ham.T-Ham)
    print("# \sum |Ham-Ham.T|",np.sum(np.abs(Ham.T-Ham)))
#    for i in range(jmax):
#        for j in range(jmax):
#            if np.abs(Ham[i,j]) > 1e-6:
#                print(i,j,Ham[i,j])
    end = time.time()
    print("## time make Hamiltonian",end-start)
    print()

    start = time.time()
    ene, vec = scipy.linalg.eigh(Ham)
    idx = np.argsort(ene)
    ene = ene[idx]
    vec = vec[:,idx]
    print("# ene",*ene[0:np.min([jmax,5])].flatten())
    #print("# vec",vec[:,0:np.min([jmax,5])])
    end = time.time()
    print("## time diagonalization",end-start)
    print()

In [22]:
Ns = 4 # number of sites
Na = 4 # number of atoms
nmax = 2 # |n> = |0>, |1>, ... |nmax> ((nmax+1) states)
U = 10.0
J = 1.0
calculate_1d_full_diag(Ns,Na,nmax,U,J)

# Ns,Na,nmax 4 4 2
# cnkc [[ 1  0  0  0  0]
 [ 1  1  1  0  0]
 [ 1  2  3  2  1]
 [ 1  3  6  7  6]
 [ 1  4 10 16 19]]
# total Hilbert space size 19
## time build_pascal 0.0007369518280029297

# no_U 4
# val_U [10. 10. 10. 10.]
# no_J 4
# pair_J [[1 2 3 1]
 [2 3 4 4]]
# val_J [1. 1. 1. 1.]
## time make_parameters_1d 0.3202192783355713

# \sum |Ham-Ham.T| 0.0
## time make Hamiltonian 1.0791900157928467

# ene -1.6710180651096458 6.475833401280539 6.698211032920437 6.698211032920441 8.20651324265321
## time diagonalization 0.0020232200622558594



In [23]:
def ham_to_vec_wave_vector(lv,cnkc,lc,nc,nmax,no_U,val_U,no_J,pair_J,val_J):
    @jit(nopython=True)
    def get_vec(v1,v0,bis,bis2,ind,ind2): ## v0: new output, v1: old input
        for i in range(1,lv+1): # state loop
#            v0[i-1] = 0.0 + 0.0j
            v0[i-1] = 0.0
            ind[0] = i
            in2bshort_bs(bis,ind,cnkc,lc,nc)
            for j in range(1,no_U+1): # site loop
                nj = np.sum(bis==j)
                v0[i-1] += 0.5 * val_U[j-1] * nj * (nj-1.0) * v1[i-1]
            for j in range(1,no_J+1): # bond loop
                f1 = pair_J[0,j-1] in bis
                f2 = pair_J[1,j-1] in bis
                if f1:
                    calc_aiadj(bis2,bis,pair_J[0,j-1],pair_J[1,j-1],nc-1)
                    bshort2in(bis2,ind2,cnkc,lc,nc)
                    if np.sum(bis2==pair_J[1,j-1]) <= nmax:
                        v0[i-1] -= val_J[j-1] * \
                            np.sqrt(np.sum(bis==pair_J[0,j-1]) * (np.sum(bis==pair_J[1,j-1])+1)) * v1[ind2[0]-1]
                if f2:
                    calc_adiaj(bis2,bis,pair_J[0,j-1],pair_J[1,j-1],nc-1)
                    bshort2in(bis2,ind2,cnkc,lc,nc)
                    if np.sum(bis2==pair_J[0,j-1]) <= nmax:
                        v0[i-1] -= val_J[j-1] * \
                            np.sqrt((np.sum(bis==pair_J[0,j-1])+1) * np.sum(bis==pair_J[1,j-1])) * v1[ind2[0]-1]
        return v0
    return get_vec

In [24]:
#@jit(nopython=True)
def calculate_1d_linearoperator(Ns,Na,nmax,U,J):
    start = time.time()
#    Ns = 7 # number of sites
#    Na = 7 # number of atoms
#    nmax = 2 # |n> = |0>, |1>, ... |nmax> ((nmax+1) states)
    lc = Ns + 1 # number of sites + 1
    nc = Na + 1  # number of atoms + 1
    nmaxp = nmax + 1 # nmax + 1, size of local Hilbert space
    cnkc, jmax = build_pascal_truncated(lc,nc,nmaxp)
    print("# Ns,Na,nmax",Ns,Na,nmax)
    print("# cnkc",cnkc)
    print("# total Hilbert space size",jmax)
    end = time.time()
    print("## time build_pascal",end-start)
    print()

    start = time.time()
#    U = 10.0
#    J = 1.0
    no_U, val_U, no_J, pair_J, val_J = make_parameters_1d(lc,U,J)
    print("# no_U",no_U)
    print("# val_U",val_U)
    print("# no_J",no_J)
    print("# pair_J",pair_J)
    print("# val_J",val_J)
    end = time.time()
    print("## time make_parameters_1d",end-start)
    print()

    start = time.time()
    get_vec = ham_to_vec_wave_vector(jmax,cnkc,lc,nc,nmax,no_U,val_U,no_J,pair_J,val_J)
    bis = np.zeros(nc-1,dtype=np.int64)
    bis2 = np.zeros(nc-1,dtype=np.int64)
    ind = np.zeros(1,dtype=np.int64)
    ind2 = np.zeros(1,dtype=np.int64)
    #v0 = np.zeros(jmax,dtype=np.complex128)
    v0 = np.zeros(jmax,dtype=np.float64)
    Ham = scipy.sparse.linalg.LinearOperator((jmax,jmax),matvec=lambda v1: get_vec(v1,v0,bis,bis2,ind,ind2))
    end = time.time()
    print("## time make Hamiltonian",end-start)
    print()

    start = time.time()
    ene, vec = scipy.sparse.linalg.eigsh(Ham,which="SA",k=np.min([5,jmax-1]))
    idx = ene.argsort()
    ene = ene[idx]
    vec = vec[:,idx]
    print("# ene",*ene[0:np.min([jmax,5])].flatten())
    #print("# vec",vec[:,0:np.min([jmax,5])])
    end = time.time()
    print("## time diagonalization",end-start)
    print()

In [25]:
Ns = 4 # number of sites
Na = 4 # number of atoms
nmax = 2 # |n> = |0>, |1>, ... |nmax> ((nmax+1) states)
U = 10.0
J = 1.0
calculate_1d_linearoperator(Ns,Na,nmax,U,J)

# Ns,Na,nmax 4 4 2
# cnkc [[ 1  0  0  0  0]
 [ 1  1  1  0  0]
 [ 1  2  3  2  1]
 [ 1  3  6  7  6]
 [ 1  4 10 16 19]]
# total Hilbert space size 19
## time build_pascal 0.0005660057067871094

# no_U 4
# val_U [10. 10. 10. 10.]
# no_J 4
# pair_J [[1 2 3 1]
 [2 3 4 4]]
# val_J [1. 1. 1. 1.]
## time make_parameters_1d 0.0006489753723144531

## time make Hamiltonian 1.3846311569213867

# ene -1.6710180651096398 6.475833401280532 6.698211032920426 6.698211032920426 8.206513242653218
## time diagonalization 0.0028581619262695312



In [26]:
@jit(nopython=True)
def make_sparse_hamiltonian_child(lv,cnkc,lc,nc,nmax,no_U,val_U,no_J,pair_J,val_J):
    bis = np.zeros(nc-1,dtype=np.int64)
    bis2 = np.zeros(nc-1,dtype=np.int64)
    ind = np.zeros(1,dtype=np.int64)
    ind2 = np.zeros(1,dtype=np.int64)
    lstki = np.array([i for k in range(2*no_J+1) for i in range(lv)],dtype=np.int64)
    lstloc = np.zeros((2*no_J+1)*lv,dtype=np.int64)
#    lstele = np.zeros((2*no_J+1)*lv,dtype=np.complex128)
    lstele = np.zeros((2*no_J+1)*lv,dtype=np.float64)
    for i in range(1,lv+1): # state loop
        ind[0] = i
        in2bshort_bs(bis,ind,cnkc,lc,nc)
        lstloc[2*no_J*lv+(i-1)] = i-1 # diagonal localtion
        for j in range(1,no_U+1): # site loop
            nj = np.sum(bis==j)
            lstele[2*no_J*lv+(i-1)] += 0.5 * val_U[j-1] * nj * (nj-1.0) # diagonal element
#            Ham[i-1,i-1] += 0.5 * val_U[j-1] * nj * (nj-1.0)
        for j in range(1,no_J+1): # bond loop
            f1 = pair_J[0,j-1] in bis
            f2 = pair_J[1,j-1] in bis
            if f1:
                calc_aiadj(bis2,bis,pair_J[0,j-1],pair_J[1,j-1],nc-1)
                bshort2in(bis2,ind2,cnkc,lc,nc)
                if np.sum(bis2==pair_J[1,j-1]) <= nmax:
                    lstele[(j-1)*lv+(i-1)] -= val_J[j-1] * \
                        np.sqrt(np.sum(bis==pair_J[0,j-1]) * (np.sum(bis==pair_J[1,j-1])+1)) # offdiag element
                    lstloc[(j-1)*lv+(i-1)] = ind2[0]-1 # offdiag localtion
#                    Ham[i-1,ind2[0]-1] -= val_J[j-1] * \
#                        np.sqrt(np.sum(bis==pair_J[0,j-1]) * (np.sum(bis==pair_J[1,j-1])+1))
#                    print("### 1 aiadj",bis2,bis,pair_J[0,j-1],pair_J[1,j-1])
            if f2:
                calc_adiaj(bis2,bis,pair_J[0,j-1],pair_J[1,j-1],nc-1)
                bshort2in(bis2,ind2,cnkc,lc,nc)
                if np.sum(bis2==pair_J[0,j-1]) <= nmax:
                    lstele[((j-1)+no_J)*lv+(i-1)] -= val_J[j-1] * \
                        np.sqrt((np.sum(bis==pair_J[0,j-1])+1) * np.sum(bis==pair_J[1,j-1])) # offdiag element
                    lstloc[((j-1)+no_J)*lv+(i-1)] = ind2[0]-1 # offdiag localtion
#                    Ham[i-1,ind2[0]-1] -= val_J[j-1] * \
#                        np.sqrt((np.sum(bis==pair_J[0,j-1])+1) * np.sum(bis==pair_J[1,j-1]))
#                    print("### 2 adiaj",bis2,bis,pair_J[0,j-1],pair_J[1,j-1])
    return lstele, lstki, lstloc

In [27]:
def make_sparse_hamiltonian(lv,lstele,lstki,lstloc):
#    return scipy.sparse.csr_matrix((lstele,(lstki,lstloc)),shape=(lv,lv),dtype=np.complex128)
    return scipy.sparse.csr_matrix((lstele,(lstki,lstloc)),shape=(lv,lv),dtype=np.float64)

In [28]:
#@jit(nopython=True)
def calculate_1d_sparse(Ns,Na,nmax,U,J):
    start = time.time()
#    Ns = 7 # number of sites
#    Na = 7 # number of atoms
#    nmax = 2 # |n> = |0>, |1>, ... |nmax> ((nmax+1) states)
    lc = Ns + 1 # number of sites + 1
    nc = Na + 1  # number of atoms + 1
    nmaxp = nmax + 1 # nmax + 1, size of local Hilbert space
    cnkc, jmax = build_pascal_truncated(lc,nc,nmaxp)
    print("# Ns,Na,nmax",Ns,Na,nmax)
    print("# cnkc",cnkc)
    print("# total Hilbert space size",jmax)
    end = time.time()
    print("## time build_pascal",end-start)
    print()

    start = time.time()
#    U = 10.0
#    J = 1.0
    no_U, val_U, no_J, pair_J, val_J = make_parameters_1d(lc,U,J)
    print("# no_U",no_U)
    print("# val_U",val_U)
    print("# no_J",no_J)
    print("# pair_J",pair_J)
    print("# val_J",val_J)
    end = time.time()
    print("## time make_parameters_1d",end-start)
    print()

    start = time.time()
    lstele, lstki, lstloc = make_sparse_hamiltonian_child(jmax,cnkc,lc,nc,nmax,no_U,val_U,no_J,pair_J,val_J)
    Ham = make_sparse_hamiltonian(jmax,lstele,lstki,lstloc)
#    print(Ham)
    end = time.time()
    print("## time make Hamiltonian",end-start)
    print()

    start = time.time()
    ene, vec = scipy.sparse.linalg.eigsh(Ham,which="SA",k=np.min([5,jmax-1]))
    idx = ene.argsort()
    ene = ene[idx]
    vec = vec[:,idx]
    print("# ene",*ene[0:np.min([jmax,5])].flatten())
    #print("# vec",vec[:,0:np.min([jmax,5])])
    end = time.time()
    print("## time diagonalization",end-start)
    print()

In [29]:
Ns = 4 # number of sites
Na = 4 # number of atoms
nmax = 2 # |n> = |0>, |1>, ... |nmax> ((nmax+1) states)
U = 10.0
J = 1.0
calculate_1d_sparse(Ns,Na,nmax,U,J)

# Ns,Na,nmax 4 4 2
# cnkc [[ 1  0  0  0  0]
 [ 1  1  1  0  0]
 [ 1  2  3  2  1]
 [ 1  3  6  7  6]
 [ 1  4 10 16 19]]
# total Hilbert space size 19
## time build_pascal 0.0007739067077636719

# no_U 4
# val_U [10. 10. 10. 10.]
# no_J 4
# pair_J [[1 2 3 1]
 [2 3 4 4]]
# val_J [1. 1. 1. 1.]
## time make_parameters_1d 0.0007109642028808594

## time make Hamiltonian 1.2164809703826904

# ene -1.6710180651096471 6.475833401280532 6.6982110329204225 6.698211032920428 8.20651324265322
## time diagonalization 0.0012171268463134766

